<a href="https://colab.research.google.com/github/shbnm93/Data-Science-Projects/blob/main/2_Eksik_Veri_Doldurma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Verilen  tablo.xlsx dosyasında eksik veri ve eksiksiz veri tabloları bulunmaktadır. Eksik veri tablosundaki eksik değerleri doldurmak için Python programlama dilinde bir kod yazınız. Kodunuz eksik değerleri doldurmak için hangi stratejiyi kullandığınızı ve bu stratejinin nedenini de açıklayınız.
Ardından, doldurulmuş veri tablosunu eksiksiz veri tablosuyla karşılaştırarak doldurma işleminin etkinliğini değerlendiriniz. Hangi metrikleri kullanarak doldurma işleminin başarısını ölçersiniz ve sonuçları nasıl yorumlarsınız?


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
# Read the data into a Pandas DataFrame
data = pd.read_excel('missed.xlsx', header=None)

In [3]:
df = pd.DataFrame(data)
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,Saat/Gün,1,2.0,3,4,5,6,7,8,9.0,...,22.0,23,24,25,26,27,28,29,30,31
1,00-01,0,0.0,80,80,0,80,80,0,80.0,...,80.0,0,0,0,0,80,80,40,0,80
2,01-02,80,80.0,0,0,0,0,0,80,0.0,...,0.0,0,80,80,80,0,0,10,0,0
3,02-03,0,0.0,0,0,80,80,0,0,0.0,...,80.0,80,0,0,0,80,80,13,80,80
4,03-04,0,80.0,80,80,0,0,80,0,80.0,...,0.0,0,80,0,80,0,0,80,0,0
5,04-05,80,0.0,0,0,80,0,0,80,0.0,...,80.0,80,0,80,0,0,80,0,80,80
6,05-06,0,0.0,80,80,0,80,80,0,80.0,...,0.0,0,0,0,0,80,80,80,720,880
7,06-07,2000,1360.0,80,1280,1760,1440,1520,1440,1040.0,...,2640.0,1760,1440,2160,2880,2400,12320,11200,19120,27200
8,07-08,25200,21280.0,4560,27840,29760,25200,27120,27200,21920.0,...,38480.0,26400,41120,34320,38720,16800,63120,66000,85440,80560
9,08-09,76960,NaN,32320,94320,97920,94560,90960,92160,71840.0,...,106800.0,56400,100800,99520,70000,50800,128080,91360,134320,79680


In [4]:
transposed_df = df.transpose()

In [5]:
transposed_df.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,Saat/Gün,00-01,01-02,02-03,03-04,04-05,05-06,06-07,07-08,08-09,...,14-15,15-16,16-17,17-18,18-19,19-20,20-21,21-22,22-23,23-24
1,1,0,80,0,0,80,0,2000,25200,76960,...,29920,68800,28560,9920,160,0,80,0,0,80
2,2.0,0.0,80.0,0.0,80.0,0.0,0.0,1360.0,21280.0,NaN,...,108480.0,143840.0,50640.0,7840.0,0.0,80.0,0.0,0.0,80.0,0.0
3,3,80,0,0,80,0,80,80,4560,32320,...,168240,92960,14160,16480,240,0,80,0,80,0
4,4,80,0,0,80,0,80,1280,27840,94320,...,170480,142160,102960,31440,400,80,0,80,0,80


In [6]:
# Set the first row as column names
transposed_df.columns  = transposed_df.iloc[0]

In [7]:
transposed_df = transposed_df.drop(0) # Drop the first row after setting it as column names

In [8]:
transposed_df.head(31)

,Saat/Gün,00-01,01-02,02-03,03-04,04-05,05-06,06-07,07-08,08-09,...,14-15,15-16,16-17,17-18,18-19,19-20,20-21,21-22,22-23,23-24
1,1,0,80,0,0,80,0,2000,25200,76960,...,29920,68800,28560,9920,160,0,80,0,0,80
2,2.0,0.0,80.0,0.0,80.0,0.0,0.0,1360.0,21280.0,NaN,...,108480.0,143840.0,50640.0,7840.0,0.0,80.0,0.0,0.0,80.0,0.0
3,3,80,0,0,80,0,80,80,4560,32320,...,168240,92960,14160,16480,240,0,80,0,80,0
4,4,80,0,0,80,0,80,1280,27840,94320,...,170480,142160,102960,31440,400,80,0,80,0,80
5,5,0,0,80,0,80,0,1760,29760,97920,...,211920,92640,70640,21680,240,80,0,80,0,0
6,6,80,0,80,0,0,80,1440,25200,94560,...,195840,153920,93920,25840,160,0,80,0,80,0
7,7,80,0,0,80,0,80,1520,27120,90960,...,199360,163440,98880,24240,240,80,0,80,0,80
8,8,0,80,0,0,80,0,1440,27200,92160,...,132880,51680,58880,2240,0,0,80,0,80,0
9,9.0,80.0,0.0,0.0,80.0,0.0,80.0,1040.0,21920.0,71840.0,...,89440.0,111920.0,76880.0,15600.0,80.0,80.0,0.0,80.0,0.0,0.0
10,10,80,0,80,0,80,0,1200,24880,87840,...,192720,148320,75200,20960,80,80,0,80,0,80


In [9]:
# Set the first column as index
transposed_df = transposed_df.set_index("Saat/Gün")

In [10]:
transposed_df.head(31)

,00-01,01-02,02-03,03-04,04-05,05-06,06-07,07-08,08-09,09-10,...,14-15,15-16,16-17,17-18,18-19,19-20,20-21,21-22,22-23,23-24
Saat/Gün,,,,,,,,,,,,,,,,,,,,,
1.0,0,80,0,0,80,0,2000,25200,76960,126480,...,29920,68800,28560,9920,160,0,80,0,0,80
2.0,0.0,80.0,0.0,80.0,0.0,0.0,1360.0,21280.0,NaN,153440.0,...,108480.0,143840.0,50640.0,7840.0,0.0,80.0,0.0,0.0,80.0,0.0
3.0,80,0,0,80,0,80,80,4560,32320,41120,...,168240,92960,14160,16480,240,0,80,0,80,0
4.0,80,0,0,80,0,80,1280,27840,94320,154880,...,170480,142160,102960,31440,400,80,0,80,0,80
5.0,0,0,80,0,80,0,1760,29760,97920,160320,...,211920,92640,70640,21680,240,80,0,80,0,0
6.0,80,0,80,0,0,80,1440,25200,94560,154720,...,195840,153920,93920,25840,160,0,80,0,80,0
7.0,80,0,0,80,0,80,1520,27120,90960,150800,...,199360,163440,98880,24240,240,80,0,80,0,80
8.0,0,80,0,0,80,0,1440,27200,92160,150320,...,132880,51680,58880,2240,0,0,80,0,80,0
9.0,80.0,0.0,0.0,80.0,0.0,80.0,1040.0,21920.0,71840.0,NaN,...,89440.0,111920.0,76880.0,15600.0,80.0,80.0,0.0,80.0,0.0,0.0


To fill the missing data in the DataFrame, we can use various methods, such as forward fill (filling missing values with the preceding value), backward fill (filling missing values with the succeeding value), or interpolation (estimating missing values based on existing data points). Here, I'll use the forward fill method to fill missing values:

In [12]:
# Fill missing values using forward fill
df2 = transposed_df.ffill(axis=1)

In [13]:
df2.head(31)

,00-01,01-02,02-03,03-04,04-05,05-06,06-07,07-08,08-09,09-10,...,14-15,15-16,16-17,17-18,18-19,19-20,20-21,21-22,22-23,23-24
Saat/Gün,,,,,,,,,,,,,,,,,,,,,
1.0,0.0,80.0,0.0,0.0,80.0,0.0,2000.0,25200.0,76960.0,126480.0,...,29920.0,68800.0,28560.0,9920.0,160.0,0.0,80.0,0.0,0.0,80.0
2.0,0.0,80.0,0.0,80.0,0.0,0.0,1360.0,21280.0,21280.0,153440.0,...,108480.0,143840.0,50640.0,7840.0,0.0,80.0,0.0,0.0,80.0,0.0
3.0,80.0,0.0,0.0,80.0,0.0,80.0,80.0,4560.0,32320.0,41120.0,...,168240.0,92960.0,14160.0,16480.0,240.0,0.0,80.0,0.0,80.0,0.0
4.0,80.0,0.0,0.0,80.0,0.0,80.0,1280.0,27840.0,94320.0,154880.0,...,170480.0,142160.0,102960.0,31440.0,400.0,80.0,0.0,80.0,0.0,80.0
5.0,0.0,0.0,80.0,0.0,80.0,0.0,1760.0,29760.0,97920.0,160320.0,...,211920.0,92640.0,70640.0,21680.0,240.0,80.0,0.0,80.0,0.0,0.0
6.0,80.0,0.0,80.0,0.0,0.0,80.0,1440.0,25200.0,94560.0,154720.0,...,195840.0,153920.0,93920.0,25840.0,160.0,0.0,80.0,0.0,80.0,0.0
7.0,80.0,0.0,0.0,80.0,0.0,80.0,1520.0,27120.0,90960.0,150800.0,...,199360.0,163440.0,98880.0,24240.0,240.0,80.0,0.0,80.0,0.0,80.0
8.0,0.0,80.0,0.0,0.0,80.0,0.0,1440.0,27200.0,92160.0,150320.0,...,132880.0,51680.0,58880.0,2240.0,0.0,0.0,80.0,0.0,80.0,0.0
9.0,80.0,0.0,0.0,80.0,0.0,80.0,1040.0,21920.0,71840.0,71840.0,...,89440.0,111920.0,76880.0,15600.0,80.0,80.0,0.0,80.0,0.0,0.0


Another method to fill missing values is interpolation, where missing values are estimated based on the existing data points. Here's how we can use linear interpolation to fill missing values in our DataFrame: